# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

# Задание 1

In [36]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

KEYWORDS = ['парсинг', 'python', 'парсить', 'стать']

In [37]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

posts = soup.find_all('article', class_='post post_preview')
posts

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/Sharku_Ogh/" title="Автор публикации">
 <span class="default-image_mini default-image_pink">
 <svg aria-hidden="true" class="icon-svg" height="24" role="img" version="1.1" viewbox="0 0 24 24" width="24"><path d="M21.5 24h-19c-1.379 0-2.5-1.122-2.5-2.5v-19c0-1.379 1.122-2.5 2.5-2.5h19c1.379 0 2.5 1.122 2.5 2.5v19c0 1.379-1.122 2.5-2.5 2.5zm-19-23c-.827 0-1.5.673-1.5 1.5v19c0 .827.673 1.5 1.5 1.5h19c.827 0 1.5-.673 1.5-1.5v-19c0-.827-.673-1.5-1.5-1.5h-19zM15.598 12.385zM19.438 15.417l-.002-.005v-.001c-.875-2.226-2.484-3.054-3.445-3.549l-.273-.143c.029-.497-.025-1.034-.167-1.599l-.128.032.123-.044c-.765-2.152-1.757-2.585-2.632-2.967l-.006-.003-.535-2.121c.357-.065.628-.375.628-.752.001-.423-.342-.765-.765-.765s-.766.342-.766.765c0 .358.248.657.581.74l-.825 1.654-.014-.003-.024-.003c-1.053-.033-1.842.369-2.5.947-.633-.322-1.515-.729-2.158-1.814.1

In [44]:
title_all = []
post_time_all = []
post_link_all = []

for post in posts:
    hubs = post.find_all('div', class_="post__text post__text-html post__text_v2")
    for hub in hubs:
        hub_lower = hub.text.lower()
        for word in KEYWORDS:
            if word in hub_lower:
                title_element = post.find('a', class_='post__title_link')
                post_time = post.find('span', class_='post__time').text
                post_time_all += [post_time]
                title_all += [title_element.text]
                post_link_all += [title_element.attrs.get('href')]

kw = {'post time': pd.Series(post_time_all), 'title': pd.Series(title_all), 'link' : pd.Series(post_link_all)}
pd.DataFrame(kw)

,post time,title,link
0,сегодня в 23:05,Восстание игроков: о главном принципе Искусств...,https://habr.com/ru/post/546308/
1,сегодня в 19:32,Как покупать и продавать валюту выгодно: плюсы...,https://habr.com/ru/post/546878/
2,сегодня в 19:07,Погружаемся в логические свойства CSS,https://habr.com/ru/company/skillfactory/blog/...
3,сегодня в 17:34,Обязательная перепись бизнеса в РФ 2021 для ИП...,https://habr.com/ru/post/546858/


# Задание 2

In [202]:
import json
import time
import re

In [631]:
EMAILS = ['xxx@yandex.ru', 'tasey14@gmail.com']
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
Headers = {'Vaar-Version': '0', 
           'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
           'Vaar-Header-App-Product': 'hackcheck-web-avast',
           'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
           'Vaar-Header-App-Build-Version': '1.0.0'
          }

In [632]:
df = pd.DataFrame()

for email in EMAILS:
    params = {'emailAddresses' : [email]}
    req_avast = requests.post(URL, json=params, headers=Headers)
    info_1 = req_avast.text
    site = re.findall(r'site":"([^"]+)', info_1)
    data = re.findall(r'publishDate":"(\d{4}-\d{2}-\d{2})', info_1)
    description = re.findall(r'description":"([^"]+)', info_1)
    req_breashed = {'почта': (email),
            'дата утечки': pd.Series(data),
            'источник утечки' : pd.Series(site),
            'описание утечки' : pd.Series(description)}
    info_2 = pd.DataFrame(req_breashed)
    print(info_2)
    df.append(info_2, ignore_index = True)
    time.sleep(0.3)
df

            почта дата утечки   источник утечки  \
0   xxx@yandex.ru  2019-03-28  verifications.io   
1   xxx@yandex.ru  2020-05-21            vk.com   
2   xxx@yandex.ru  2020-05-21    bitrewards.com   
3   xxx@yandex.ru  2020-12-10          job12.eu   
4   xxx@yandex.ru  2020-12-03    worldofauto.ru   
5   xxx@yandex.ru  2020-04-09         vkmix.com   
6   xxx@yandex.ru  2020-12-17           e-ga.ru   
7   xxx@yandex.ru  2020-12-17        tcl.y17.su   
8   xxx@yandex.ru  2020-11-05      vimeworld.ru   
9   xxx@yandex.ru  2020-12-10        shedevr.ru   
10  xxx@yandex.ru  2019-05-23   livejournal.com   

                                      описание утечки  
0   Big data e-mail verification platform verifica...  
1   At some time in 2020, the Russian social netwo...  
2   In April 2020, the e-commerce platform BitRewa...  
3   In November 2020, a collection of over 23,000 ...  
4   In November 2020, a collection of over 23,000 ...  
5   At an unconfirmed date, the Russian social med.

""


In [3]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {'Vaar-Version': '0', 'Vaar-Header-App-Product': 'hackcheck-web-awast', 'Vaar-Header-App-Product-Name': 'hackcheck-web-avast', 'Vaar-Header-App-Build-Version': '1.0.0'}
addresses = {'emailAddresses': EMAIL}
res = requests.post(url, json=addresses, headers=headers)
df = pd.DataFrame(res.json()['breaches']).T.reset_index()
df = df[['site', 'description', 'publishDate']]
df['email'] = [', '.join(el.keys()) for el in list(res.json()['data'].values())]

ValueError: Length of values does not match length of index